# Preprocessing data for Task 1
## Import section

In [1]:
import pandas as pd
import numpy as np
from copy import copy

## Load all tables onto dataframes

In [2]:
train_t_1 = pd.read_csv("data/phase1_training/20min_avg_travel_time_training_phase1.csv")
train_t_2 = pd.read_csv("data/phase1_training/20min_avg_volume_training_phase1.csv")
train_t_3 = pd.read_csv("data/road/links_table3.csv")
train_t_4 = pd.read_csv("data/road/routes_table 4.csv")
train_t_5 = pd.read_csv("data/phase1_training/trajectories_training_phase1_table5.csv")
train_t_6 = pd.read_csv("data/phase1_training/volume_training_phase1_table6.csv")
train_t_7 = pd.read_csv("data/weather/weather_July_01_Oct_17_table7.csv")

In [3]:
sample_data_final = pd.read_csv("data/submission_sample/submission_sample_travelTime.csv")

In [4]:
test_t_1 = pd.read_csv("data/phase1_test/20min_avg_travel_time_test_phase1.csv")
test_t_2 = pd.read_csv("data/phase1_test/20min_avg_volume_test_phase1.csv")
test_t_5 = pd.read_csv("data/phase1_test/trajectories_test_phase1_table5.csv")
test_t_6 = pd.read_csv("data/phase1_test/volume_test_phase1_table6.csv")
test_t_7 = pd.read_csv("data/weather/weather_Oct_18_Oct_24_table7.csv")

## Removing outliers from table 5 -- trajectory data

In [5]:
for k, row in train_t_5.iterrows():
    if row['travel_time'] > 600:
        last_value = train_t_5.loc[k-1,'travel_time']
        next_value = train_t_5.loc[k+1,'travel_time']
        if last_value < 600:
            train_t_5.loc[k, 'travel_time'] = (last_value + next_value)/2.0
        else:
            train_t_5.loc[k, 'travel_time'] = last_value

In [6]:
for k, row in test_t_5.iterrows():
    if row['travel_time'] > 600:
        last_value = test_t_5.loc[k-1,'travel_time']
        next_value = test_t_5.loc[k+1,'travel_time']
        if last_value < 600:
            test_t_5.loc[k, 'travel_time'] = (last_value + next_value)/2.0
        else:
            test_t_5.loc[k, 'travel_time'] = last_value

## Grouping the travel time of individual into average travel time for every 20 mins time window

In [7]:
# processing training data -- table 5

train_t_5['starting_time'] = pd.to_datetime(train_t_5['starting_time'], format='%Y-%m-%d %H:%M:%S')

train_t_5 = train_t_5.set_index(['starting_time'])

train_t_5 = train_t_5.groupby([pd.Grouper(freq='20Min'), 
                               'intersection_id', 
                               'tollgate_id']).travel_time.mean().reset_index().rename(
    columns={'travel_time':'average_travl_time'})

# processing test data -- table 5

test_t_5['starting_time'] = pd.to_datetime(test_t_5['starting_time'], format="%Y-%m-%d %H:%M:%S")

test_t_5 = test_t_5.set_index(['starting_time'])

test_t_5 = test_t_5.groupby([pd.Grouper(freq='20Min'), 
                             'intersection_id', 
                             'tollgate_id']).travel_time.mean().reset_index().rename(
    columns={'travel_time':'average_travl_time'})

print(train_t_5.shape, test_t_5.shape)

(25144, 4) (448, 4)


## create unique pairs of all intersection and toll

In [8]:
all_toll_intersections = []

for j in range(sample_data_final.shape[0]):    
    intersection=sample_data_final.loc[j]['intersection_id']
    tollgate=sample_data_final.loc[j]['tollgate_id']
    token = (intersection,tollgate)
    if token not in all_toll_intersections:
        all_toll_intersections.append(token)
        
sample_time = []
sample_times = sample_data_final[(sample_data_final['tollgate_id']==1)&
                                 (sample_data_final['intersection_id']=='B')]['time_window']

for st in sample_times:
    sample_time.append(pd.to_datetime(st.split(',')[0][1:], format="%Y-%m-%d %H:%M:%S") - pd.DateOffset(hours=2))

sample_time = pd.Series(sample_time).values

## method to replace missing time

In [9]:
def replace_missing_time(test, tollgate, intersection, iteration, sample_time):
    while iteration > 0:
        try:
            missing_time = test[(test['tollgate_id']==tollgate)& 
                                (test['starting_time'] == sample_time[iteration - 1])& 
                                (test['intersection_id']==intersection)]['average_travl_time']
            
            return missing_time.values[0]
        except Exception as e:
            iteration = iteration - 1
            continue

## replace the missing time in table 5

In [10]:

for intersection, tollgate in all_toll_intersections:
    test_toll_intersections = copy(test_t_5[(test_t_5['tollgate_id']==tollgate) & 
                                                       (test_t_5['intersection_id']==intersection) 
                                                     ].reset_index()) 
    
    test_time= test_t_5[(test_t_5['tollgate_id']==tollgate) & 
                                   (test_t_5['intersection_id']==intersection)
                                  ]['starting_time'].values
    
    test_toll_intersections.drop('index',axis=1,inplace=True)
    test_toll_intersections = test_toll_intersections.loc[0]
    
    for k in range(len(sample_time)):
        if sample_time[k] not in test_time: 
            test_toll_intersections['starting_time'] = sample_time[k]
            test_toll_intersections['average_travl_time'] = replace_missing_time(test_t_5, 
                                                                                 tollgate, 
                                                                                 intersection,
                                                                                 k, 
                                                                                 sample_time)
            
            test_t_5 = test_t_5.append(test_toll_intersections)
            
test_t_5 = test_t_5.reset_index()
test_t_5.drop('index', axis=1, inplace=True)

In [11]:
train_t_5 = train_t_5.append(test_t_5)

In [12]:
train_t_5['lag1'] = train_t_5['average_travl_time'].shift(1)
train_t_5['lag2'] = train_t_5['average_travl_time'].shift(2)
train_t_5['lag3'] = train_t_5['average_travl_time'].shift(3)
train_t_5['lag4'] = train_t_5['average_travl_time'].shift(4)
train_t_5['lag5'] = train_t_5['average_travl_time'].shift(5)
train_t_5['lag6'] = train_t_5['average_travl_time'].shift(6)
train_t_5['lag7'] = train_t_5['average_travl_time'].shift(7)

## create a heat map of the table 5

In [13]:
import seaborn as sns
sns.heatmap(train_t_5.corr(), annot = True, fmt = ".2f")
display()

## create another dataframe from table 5 with a date offset of 2 hours 

In [14]:
test_t_5['starting_time'] = test_t_5['starting_time'] + pd.DateOffset(hours=2)
test_t_5_dup = test_t_5
test_t_5_dup.drop('average_travl_time', axis=1, inplace=True)

In [15]:
test_t_5_dup.head()

,starting_time,intersection_id,tollgate_id
0,2016-10-18 08:00:00,A,2
1,2016-10-18 08:00:00,A,3
2,2016-10-18 08:00:00,B,1
3,2016-10-18 08:00:00,B,3
4,2016-10-18 08:00:00,C,1


In [16]:
print(test_t_5_dup.shape)
print(train_t_5.shape)

(504, 3)
(25648, 11)


## Additional info -- adding festival day's data

In [17]:
# Chinese festival days

from datetime import datetime

start_date = datetime(2016, 9, 15)
end_date = datetime(2016, 9, 17)
holiday_range = pd.date_range(start_date, end_date)
start_date2 = datetime(2016, 10, 1)
end_date2 = datetime(2016, 10, 7)
holiday_range= holiday_range.append(pd.date_range(start_date2, end_date2))

In [18]:
# Adding Extra column with the name ch_holidays. If the date exists in between Holiday range,the value of 
#the column will be 1 or else 0

def identify_holiday_dates(data, start_time):
    day_of_the_week = pd.get_dummies(data[start_time].dt.weekday_name)
    hr_of_the_day = pd.get_dummies(data[start_time].dt.hour, prefix='hour_')
    minute= pd.get_dummies(data[start_time].dt.minute)
    data = pd.concat([data, day_of_the_week, hr_of_the_day, minute], axis=1)
    data['date'] = data[start_time].dt.date
    data['date'] = data['date'].astype(str)
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
    data['hour'] = data[start_time].dt.hour.astype(int)
    start_time_date = data[start_time].dt.date
    
    for k, row in data.iterrows():
        data.loc[k,"ch_holidays"] = 0
        if start_time_date.loc[k] in holiday_range: data.loc[k, "ch_holidays"] = 1
    return data

In [19]:
train_t_5 = identify_holiday_dates(train_t_5, "starting_time")
test_t_5_dup = identify_holiday_dates(test_t_5_dup, "starting_time")

In [20]:
print(test_t_5_dup.shape)
train_t_5.head()

(504, 20)


,starting_time,intersection_id,tollgate_id,average_travl_time,lag1,lag2,lag3,lag4,lag5,lag6,...,hour__20,hour__21,hour__22,hour__23,0,20,40,date,hour,ch_holidays
0,2016-07-19 00:00:00,B,3,70.85,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,2016-07-19,0,0.0
1,2016-07-19 00:20:00,A,2,58.05,70.85,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,2016-07-19,0,0.0
2,2016-07-19 00:20:00,B,1,79.76,58.05,70.85,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,2016-07-19,0,0.0
3,2016-07-19 00:20:00,B,3,148.79,79.76,58.05,70.85,NaN,NaN,NaN,...,0,0,0,0,0,1,0,2016-07-19,0,0.0
4,2016-07-19 00:40:00,B,1,137.98,148.79,79.76,58.05,70.85,NaN,NaN,...,0,0,0,0,0,0,1,2016-07-19,0,0.0


## Adding table 7 (weather data)

In [21]:
# loading and appending weather test data to weather train data

train_t_7 = train_t_7.append(test_t_7).reset_index()
train_t_7['date'] = pd.to_datetime(train_t_7['date'], format='%Y-%m-%d')

## remove outlier from weather table

In [22]:
# replacing the outlier value of 99017 in wind_direction of weatherData by avg of previous and next value
# 99017 is the only outlier

for i, row in train_t_7.iterrows():
    if row['wind_direction']== 999017.0:
        previous_value = train_t_7.loc[i-1,'wind_direction']
        next_value = train_t_7.loc[i+1,'wind_direction']
        if next_value != 999017.0:
            train_t_7.loc[i, 'wind_direction'] = (previous_value + next_value)/2.0
        else:
            train_t_7.loc[i, 'wind_direction'] = previous_value

In [23]:
train_t_7.head()

,index,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,0,2016-07-01,0,1000.4,1005.3,225.0,2.1,26.4,94.0,0.0
1,1,2016-07-01,3,1000.5,1005.3,187.0,2.7,29.0,76.0,0.0
2,2,2016-07-01,6,998.9,1003.7,212.0,2.9,31.7,67.0,0.0
3,3,2016-07-01,9,998.7,1003.5,244.0,2.7,31.6,59.0,0.0
4,4,2016-07-01,12,999.7,1004.5,222.0,1.3,29.9,68.0,0.0


In [24]:
train_t_5.shape

(25648, 48)

## Combining table 5 with table 7 (trajectory data + weather data)

In [25]:
# Turn hour into 3 hour intervals and then combine with weather data

def addWeatherData(df):
    for i, row in df.iterrows():
        if row['hour'] in [23,0,1]: df.loc[i, "hour"] = 0
        elif row['hour'] in [2,3,4]: df.loc[i, "hour"] = 3 
        elif row['hour'] in [5,6,7]: df.loc[i, "hour"] = 6         
        elif row['hour'] in [8,9,10]: df.loc[i, "hour"] = 9         
        elif row['hour'] in [11,12,13]: df.loc[i, "hour"] = 12         
        elif row['hour'] in [14,15,16]: df.loc[i, "hour"] = 15         
        elif row['hour'] in [17,18,19]: df.loc[i, "hour"] = 18         
        elif row['hour'] in [20,21,22]: df.loc[i, "hour"] = 21
    return pd.merge(df, train_t_7, on =['date', 'hour'], how='left')

In [26]:
train_t_5 = addWeatherData(train_t_5)
test_t_5_dup = addWeatherData(test_t_5_dup)

In [27]:
train_t_5.head()

,starting_time,intersection_id,tollgate_id,average_travl_time,lag1,lag2,lag3,lag4,lag5,lag6,...,hour,ch_holidays,index,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-07-19 00:00:00,B,3,70.85,NaN,NaN,NaN,NaN,NaN,NaN,...,6,0.0,146.0,999.7,1004.5,239.0,1.9,31.8,64.0,0.0
1,2016-07-19 00:20:00,A,2,58.05,70.85,NaN,NaN,NaN,NaN,NaN,...,6,0.0,146.0,999.7,1004.5,239.0,1.9,31.8,64.0,0.0
2,2016-07-19 00:20:00,B,1,79.76,58.05,70.85,NaN,NaN,NaN,NaN,...,6,0.0,146.0,999.7,1004.5,239.0,1.9,31.8,64.0,0.0
3,2016-07-19 00:20:00,B,3,148.79,79.76,58.05,70.85,NaN,NaN,NaN,...,6,0.0,146.0,999.7,1004.5,239.0,1.9,31.8,64.0,0.0
4,2016-07-19 00:40:00,B,1,137.98,148.79,79.76,58.05,70.85,NaN,NaN,...,6,0.0,146.0,999.7,1004.5,239.0,1.9,31.8,64.0,0.0


In [28]:
train_t_5 = train_t_5.drop(['hour','date'],axis=1)
test_t_5_dup = test_t_5_dup.drop(['hour','date'],axis=1)
train_t_5.shape

(25648, 54)

## Processing table 4

In [29]:
divison_row = []

def divide(seq):
    return seq.split(',')

train_t_4.link_seq = train_t_4.link_seq.apply(divide)

X = train_t_4.apply(lambda row: [divison_row.append([row['intersection_id'], row['tollgate_id'], link]) 
                         for link in row.link_seq], axis=1)

table_headers = ['intersection_id', 'tollgate_id', 'link_id']
train_t_4_new= pd.DataFrame(divison_row, columns=table_headers)
train_t_4_new['link_id'] = train_t_4_new['link_id'].astype(str)

## Processing table 3

In [30]:
train_t_3['cross_in'] = 0
train_t_3['cross_out'] = 0

for k, row in train_t_3.iterrows():
    if ',' in str(row['out_top']):
        train_t_3.loc[k, 'cross_out'] = 1
    if ',' in str(row['in_top']):
        train_t_3.loc[k, 'cross_in'] = 1
        
train_t_3['link_id'] = train_t_3['link_id'].astype(str)  
train_t_4_new = pd.merge(train_t_4_new, train_t_3, on='link_id', how='left')
train_t_4_new.drop(['in_top', 'out_top'], axis=1, inplace=True)

## Merging table 4 with table 5

In [31]:
join_incount= train_t_4_new[['intersection_id', 'tollgate_id', 'cross_in']].groupby([
    'intersection_id', 'tollgate_id']).cross_in.sum().reset_index().rename(columns={
    'cross_in':'inlink_cross_count'})

join_outcount = train_t_4_new[['intersection_id', 'tollgate_id', 'cross_out']].groupby([
    'intersection_id', 'tollgate_id']).cross_out.sum().reset_index().rename(columns={
    'cross_out':'outlink_cross_count'})

final = pd.merge(join_incount, join_outcount, on=['intersection_id', 'tollgate_id'], how='left')

_len = train_t_4_new[['intersection_id', 'tollgate_id', 'length']].groupby([
    'intersection_id', 'tollgate_id']).length.sum().reset_index()

final = pd.merge(final, _len, on=['intersection_id', 'tollgate_id'], how='left')

link_count = train_t_4_new[['intersection_id', 'tollgate_id']].groupby([
    'intersection_id', 'tollgate_id']).size().reset_index().rename(columns={0:'link_count'})

final = pd.merge(final, link_count, on=['intersection_id', 'tollgate_id'], how='left')

lane1_length = train_t_4_new[train_t_4_new.lanes==1][['intersection_id', 'tollgate_id', 'length']].groupby([
    'intersection_id', 'tollgate_id']).length.sum().reset_index().rename(columns={'length':'lane1_length'})

final = pd.merge(final, lane1_length, on=['intersection_id', 'tollgate_id'], how='left')

lane1_count = train_t_4_new[train_t_4_new.lanes== 1][['intersection_id', 'tollgate_id']].groupby([
    'intersection_id', 'tollgate_id']).size().reset_index().rename(columns = {0:'lane1_count'})

final = pd.merge(final, lane1_count, on =['intersection_id', 'tollgate_id'], how='left')

lane2_length = train_t_4_new[train_t_4_new.lanes==2][['intersection_id', 'tollgate_id', 'length']].groupby([
    'intersection_id', 'tollgate_id']).length.sum().reset_index().rename(columns={'length':'lane2_length'})

final = pd.merge(final, lane2_length, on=['intersection_id', 'tollgate_id'], how='left')

lane2_count = train_t_4_new[train_t_4_new.lanes== 2][['intersection_id', 'tollgate_id']].groupby([
    'intersection_id', 'tollgate_id']).size().reset_index().rename(columns = {0:'lane2_count'})

final = pd.merge(final, lane2_count, on =['intersection_id', 'tollgate_id'], how='left')

lane3_length = train_t_4_new[train_t_4_new.lanes==3][['intersection_id', 'tollgate_id', 'length']].groupby([
    'intersection_id', 'tollgate_id']).length.sum().reset_index().rename(columns={'length':'lane3_length'})

final = pd.merge(final, lane3_length, on=['intersection_id', 'tollgate_id'], how='left')

lane3_count = train_t_4_new[train_t_4_new.lanes==3][['intersection_id', 'tollgate_id']].groupby([
    'intersection_id', 'tollgate_id']).size().reset_index().rename(columns = {0:'lane3_count'})

final = pd.merge(final,lane3_count,on =['intersection_id', 'tollgate_id'] ,how='left')

lane4_length = train_t_4_new[train_t_4_new.lanes==4][['intersection_id', 'tollgate_id', 'length']].groupby([
    'intersection_id', 'tollgate_id']).length.sum().reset_index().rename(columns={'length':'lane4_length'})

final = pd.merge(final, lane4_length, on=['intersection_id', 'tollgate_id'],how='left')

lane4_count = train_t_4_new[train_t_4_new.lanes==4][['intersection_id', 'tollgate_id']].groupby([
    'intersection_id', 'tollgate_id']).size().reset_index().rename(columns = {0:'lane4_count'})

final = pd.merge(final,lane4_count,on =['intersection_id', 'tollgate_id'] ,how='left')

final.fillna(0, inplace=True)

train_t_5 = pd.merge(train_t_5, final, on=['intersection_id', 'tollgate_id'], how='left')

test_t_5_dup = pd.merge(test_t_5_dup, final, on=['intersection_id', 'tollgate_id'], how='left')

## Formatting starting and end time

In [32]:
def time_period(data, start_time, end_time):
    st = data[start_time].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
    et = data[end_time].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
    data['time_window'] = '[' + st + ',' + et + ')'
    
    return data.drop([start_time, end_time], axis=1)

In [33]:
test_t_5_dup['end'] = test_t_5_dup['starting_time'] + pd.DateOffset(minutes=20)
train_t_5['end'] = train_t_5['starting_time'] + pd.DateOffset(minutes=20)
test_t_5_dup = time_period(test_t_5_dup, 'starting_time', 'end')
train_t_5 = time_period(train_t_5, 'starting_time', 'end')

In [34]:
test_t_5_dup = test_t_5_dup.set_index(['intersection_id', 'tollgate_id', 'time_window'])
train_t_5 = train_t_5.set_index(['intersection_id','tollgate_id','time_window'])

In [35]:
train_t_5.shape

(25648, 63)

In [36]:
test_t_5_columns, train_t_5_columns = list(test_t_5_dup.columns.values), list(train_t_5.columns.values)

In [37]:
m_t5 =  [data for data in train_t_5_columns if data not in test_t_5_columns]

In [38]:
for label in m_t5:
    test_t_5_dup[label] = 0

In [39]:
test_t_5_dup = test_t_5_dup[train_t_5_columns]

## Handling the missing values with mean value in table 5

In [40]:
def fill_nullvalues(data):
    return data.fillna(data.mean())

test_t_5_dup = fill_nullvalues(test_t_5_dup)

train_t_5 =fill_nullvalues(train_t_5)

In [41]:
print(test_t_5_dup.shape, train_t_5.shape)

(504, 63) (25648, 63)


In [42]:
test_t_5_dup.to_csv("data/preprocessing_data/task1_preprocess_test_data.csv")
train_t_5.to_csv("data/preprocessing_data/task1_preprocess_training_data.csv")

In [43]:
train_t_5.columns.values

array(['average_travl_time', 'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
       'lag6', 'lag7', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', 'hour__0', 'hour__1',
       'hour__2', 'hour__3', 'hour__4', 'hour__5', 'hour__6', 'hour__7',
       'hour__8', 'hour__9', 'hour__10', 'hour__11', 'hour__12',
       'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17',
       'hour__18', 'hour__19', 'hour__20', 'hour__21', 'hour__22',
       'hour__23', 0, 20, 40, 'ch_holidays', 'index', 'pressure',
       'sea_pressure', 'wind_direction', 'wind_speed', 'temperature',
       'rel_humidity', 'precipitation', 'inlink_cross_count',
       'outlink_cross_count', 'length', 'link_count', 'lane1_length',
       'lane1_count', 'lane2_length', 'lane2_count', 'lane3_length',
       'lane3_count', 'lane4_length', 'lane4_count'], dtype=object)

In [44]:
test_t_5_dup.shape

(504, 63)